In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.constants import TARGET_MAX_LENGHT, MAX_LENGHT_SOURCE
from src.data_utils.dataset import build_datset_train_val, VOCAB_SIZE, LHAND_IDX, LHAND_IDX, start_token_idx, end_token_idx, pre_process, pad_token_idx, FEATURE_COLUMNS
from src.prod_models.builder import build_prod_transformer_model_v2
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf
import numpy as np

TRIALS = 200
EPOCHS = 5000
EPOCHS_PER_TRIAL = 15
BATCH_SIZE = 128
TRAIN_SPLIT = 0.8
MODEL_NAME = "prod_v2"

In [2]:
train_dataset, val_dataset = build_datset_train_val(split=TRAIN_SPLIT, batch_size=BATCH_SIZE)

train split: 28160 | val split: 6656


In [3]:
def objective(trial):
    tf.keras.backend.clear_session()
    model = build_prod_transformer_model_v2(trial=trial)
    model.build([(None, MAX_LENGHT_SOURCE, int(FEATURE_COLUMNS.shape[0]/2)), (None, TARGET_MAX_LENGHT)])
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_PER_TRIAL, callbacks=get_predefine_callbacks(model_name=MODEL_NAME, patience=3), verbose=0)
    levenshtein = model.evaluate(val_dataset)[-1]

    return  levenshtein

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=TRIALS, gc_after_trial=True, show_progress_bar=True)

[I 2023-09-01 16:10:47,910] A new study created in memory with name: no-name-4e63ebef-6817-471a-88f9-3201a07fdb0e


  0%|          | 0/200 [00:00<?, ?it/s]

77/77 [==============================] - 1s 11ms/step - loss: 0.8188 - accuracy: 0.7599 - sparse_levenshtein_v1: 0.2363
[I 2023-09-01 16:11:45,539] Trial 0 finished with value: 0.23633567988872528 and parameters: {'attention_heads': 5, 'learning_rate': 0.02496233768447454, 'drop_out_out_decoder': 0.15000000000000002, 'encoder_kernel_size': 5}. Best is trial 0 with value: 0.23633567988872528.
77/77 [==============================] - 1s 11ms/step - loss: 0.7497 - accuracy: 0.7780 - sparse_levenshtein_v1: 0.2200
[I 2023-09-01 16:13:51,208] Trial 1 finished with value: 0.2200479805469513 and parameters: {'attention_heads': 5, 'learning_rate': 5.5321478946485995e-05, 'drop_out_out_decoder': 0.15000000000000002, 'encoder_kernel_size': 12}. Best is trial 1 with value: 0.2200479805469513.
77/77 [==============================] - 1s 14ms/step - loss: 0.6586 - accuracy: 0.8022 - sparse_levenshtein_v1: 0.1959
[I 2023-09-01 16:16:25,157] Trial 2 finished with value: 0.19589154422283173 and paramet

In [5]:
trials = study.best_trials

In [6]:
trials[0]

FrozenTrial(number=78, state=TrialState.COMPLETE, values=[0.12880994379520416], datetime_start=datetime.datetime(2023, 9, 1, 18, 51, 42, 503274), datetime_complete=datetime.datetime(2023, 9, 1, 18, 53, 57, 726596), params={'attention_heads': 7, 'learning_rate': 0.0020283022893461916, 'drop_out_out_decoder': 0.15000000000000002, 'encoder_kernel_size': 9}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'attention_heads': IntDistribution(high=8, log=False, low=1, step=1), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'drop_out_out_decoder': FloatDistribution(high=0.5, log=False, low=0.0, step=0.05), 'encoder_kernel_size': IntDistribution(high=12, log=False, low=3, step=1)}, trial_id=78, value=None)

In [7]:
tf.keras.backend.clear_session()
trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")

    model = build_prod_transformer_model_v2(trial=trial)

    model.build([(None, MAX_LENGHT_SOURCE, int(FEATURE_COLUMNS.shape[0]/2)), (None, TARGET_MAX_LENGHT)])

    print(model.summary())
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=get_predefine_callbacks(model_name=MODEL_NAME, patience=10))
   
    print('validation levenshtein distance: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{MODEL_NAME}")

    print(f"Metrics in Validation: {model.evaluate(val_dataset)}")

Best model: 1
Model: "finger_spelling_v2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 landmark_embedding_v2 (Lan  multiple                  75136     
 dmarkEmbeddingV2)                                               
                                                                 
 basic_positional_embedding  multiple                  8064      
 s (BasicPositionalEmbeddin                                      
 gs)                                                             
                                                                 
 transformer_encoder (Trans  multiple                  68944     
 formerEncoder)                                                  
                                                                 
 transformer_decoder (Trans  multiple                  136406    
 formerDecoder)                                                  
                                  

In [8]:
# Save model

model.save(f"../models/{MODEL_NAME}", save_format="tf")

INFO:tensorflow:Assets written to: ../models/prod_v2/assets


INFO:tensorflow:Assets written to: ../models/prod_v2/assets


# TF lite

In [9]:
from src.export_tf_lite.model import TFLiteModel

tflitemodel_base = TFLiteModel(model)

In [10]:
tflitemodel_base(tf.random.uniform(shape=[128, 104]))

{'outputs': <tf.Tensor: shape=(18, 59), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}

In [11]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = keras_model_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpgbn00xou/assets


INFO:tensorflow:Assets written to: /tmp/tmpgbn00xou/assets


In [12]:
import json

with open("../models/model.tflite", "wb") as f:    
    f.write(tflite_model)
    
infargs = {"selected_columns" : list(FEATURE_COLUMNS)}

with open("../models/inference_args.json", "w") as json_file:
    json.dump(infargs, json_file)

In [13]:
!zip submission.zip  '../models/model.tflite' '../models/inference_args.json'

updating: ../models/model.tflite (deflated 78%)
updating: ../models/inference_args.json (deflated 84%)


# Test results

In [14]:
# from batch 1

source_batch, target_batch = next(iter(val_dataset))[0]

In [15]:
REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

interpreter = tf.lite.Interpreter("../models/model.tflite")

with open ("../data/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)

rev_character_map = {j:i for i,j in character_map.items()}
found_signatures = list(interpreter.get_signature_list().keys())

if REQUIRED_SIGNATURE not in found_signatures:
    raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner(REQUIRED_SIGNATURE)

prediction_str = ""
for source_element, target_element in zip(source_batch, target_batch):
    output = prediction_fn(inputs=source_element)

    print("generated: ", "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)]))
    print("target: ", "".join([rev_character_map.get(s, "") for s in target_element.numpy()]))

generated:  784-084-0800
target:  www.empresadelimpieza.org
generated:  885 co 10
target:  5 north 3029th road
generated:  808 dowe
target:  seller-online.net/shonanholic
generated:  958-588-1601
target:  eduardbatlle.cat
generated:  4088 dee 8849
target:  867-401-2165
generated:  9848 480
target:  35 davidoff drive
generated:  4888 484
target:  856-675-9951
generated:  458-848-2488
target:  621919 101da jimmerson
generated:  788 7849
target:  1150 apostol
generated:  778-888-8101
target:  992-961-3811
generated:  8984 848
target:  320846 skaggs st
generated:  beende con
target:  401 nw via della ct
generated:  medie
target:  3091 485
generated:  8888 888
target:  +30-1286-33-43-8907
generated:  meffree be
target:  5459 wells cemetery no 1
generated:  8888 888
target:  +32-8177-3222-69
generated:  6787 core
target:  www.driverscape.com
generated:  400-048-0001
target:  7011 henry circle
generated:  5888 co 10
target:  9365 carmel airport
generated:  858-888-3161
target:  177 kipawa cou

In [16]:
interpreter.get_input_details()

[{'name': 'serving_default_inputs:0',
  'index': 0,
  'shape': array([128,  52], dtype=int32),
  'shape_signature': array([ -1, 104], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]